In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -19.000000. running mean: -20.980000
episode 3.000000, reward total was -21.000000. running mean: -20.980200
episode 4.000000, reward total was -21.000000. running mean: -20.980398
episode 5.000000, reward total was -20.000000. running mean: -20.970594
episode 6.000000, reward total was -21.000000. running mean: -20.970888
episode 7.000000, reward total was -21.000000. running mean: -20.971179
episode 8.000000, reward total was -21.000000. running mean: -20.971467
episode 9.000000, reward total was -20.000000. running mean: -20.961753
episode 10.000000, reward total was -21.000000. running mean: -20.962135
episode 11.000000, reward total was -21.000000. running mean: -20.962514
episode 12.000000, reward total was -20.000000. running mean: -20.952889
episode 13.000000, reward total was -21.000000. running mean: -20.953360
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -20.000000. running mean: -20.704704
episode 115.000000, reward total was -20.000000. running mean: -20.697657
episode 116.000000, reward total was -20.000000. running mean: -20.690680
episode 117.000000, reward total was -21.000000. running mean: -20.693773
episode 118.000000, reward total was -20.000000. running mean: -20.686836
episode 119.000000, reward total was -20.000000. running mean: -20.679967
episode 120.000000, reward total was -21.000000. running mean: -20.683168
episode 121.000000, reward total was -21.000000. running mean: -20.686336
episode 122.000000, reward total was -20.000000. running mean: -20.679473
episode 123.000000, reward total was -21.000000. running mean: -20.682678
episode 124.000000, reward total was -19.000000. running mean: -20.665851
episode 125.000000, reward total was -19.000000. running mean: -20.649193
episode 126.000000, reward total was -20.000000. running mean: -20.642701
episode 127.000000, reward total was -

episode 225.000000, reward total was -20.000000. running mean: -20.590564
episode 226.000000, reward total was -21.000000. running mean: -20.594658
episode 227.000000, reward total was -21.000000. running mean: -20.598711
episode 228.000000, reward total was -19.000000. running mean: -20.582724
episode 229.000000, reward total was -20.000000. running mean: -20.576897
episode 230.000000, reward total was -21.000000. running mean: -20.581128
episode 231.000000, reward total was -20.000000. running mean: -20.575317
episode 232.000000, reward total was -20.000000. running mean: -20.569564
episode 233.000000, reward total was -20.000000. running mean: -20.563868
episode 234.000000, reward total was -21.000000. running mean: -20.568229
episode 235.000000, reward total was -19.000000. running mean: -20.552547
episode 236.000000, reward total was -21.000000. running mean: -20.557022
episode 237.000000, reward total was -20.000000. running mean: -20.551451
episode 238.000000, reward total was -

episode 336.000000, reward total was -18.000000. running mean: -20.184322
episode 337.000000, reward total was -21.000000. running mean: -20.192479
episode 338.000000, reward total was -20.000000. running mean: -20.190554
episode 339.000000, reward total was -20.000000. running mean: -20.188649
episode 340.000000, reward total was -20.000000. running mean: -20.186762
episode 341.000000, reward total was -21.000000. running mean: -20.194895
episode 342.000000, reward total was -21.000000. running mean: -20.202946
episode 343.000000, reward total was -21.000000. running mean: -20.210916
episode 344.000000, reward total was -20.000000. running mean: -20.208807
episode 345.000000, reward total was -19.000000. running mean: -20.196719
episode 346.000000, reward total was -19.000000. running mean: -20.184752
episode 347.000000, reward total was -19.000000. running mean: -20.172904
episode 348.000000, reward total was -19.000000. running mean: -20.161175
episode 349.000000, reward total was -

...

episode 6493.000000, reward total was -1.000000. running mean: 0.124295
episode 6494.000000, reward total was 1.000000. running mean: 0.133052
episode 6495.000000, reward total was -11.000000. running mean: 0.021721
episode 6496.000000, reward total was -1.000000. running mean: 0.011504
episode 6497.000000, reward total was 6.000000. running mean: 0.071389
episode 6498.000000, reward total was 3.000000. running mean: 0.100675
episode 6499.000000, reward total was -3.000000. running mean: 0.069668
episode 6500.000000, reward total was 7.000000. running mean: 0.138972
episode 6501.000000, reward total was -7.000000. running mean: 0.067582
episode 6502.000000, reward total was 4.000000. running mean: 0.106906
episode 6503.000000, reward total was 3.000000. running mean: 0.135837
episode 6504.000000, reward total was -1.000000. running mean: 0.124479
episode 6505.000000, reward total was 2.000000. running mean: 0.143234
episode 6506.000000, reward total was 3.000000. running mean: 0.171801

episode 6608.000000, reward total was 6.000000. running mean: 0.281177
episode 6609.000000, reward total was 7.000000. running mean: 0.348366
episode 6610.000000, reward total was 2.000000. running mean: 0.364882
episode 6611.000000, reward total was 8.000000. running mean: 0.441233
episode 6612.000000, reward total was -3.000000. running mean: 0.406821
episode 6613.000000, reward total was 7.000000. running mean: 0.472753
episode 6614.000000, reward total was -3.000000. running mean: 0.438025
episode 6615.000000, reward total was 2.000000. running mean: 0.453645
episode 6616.000000, reward total was 1.000000. running mean: 0.459108
episode 6617.000000, reward total was 10.000000. running mean: 0.554517
episode 6618.000000, reward total was -3.000000. running mean: 0.518972
episode 6619.000000, reward total was 4.000000. running mean: 0.553782
episode 6620.000000, reward total was -5.000000. running mean: 0.498245
episode 6621.000000, reward total was 5.000000. running mean: 0.543262
e

episode 6723.000000, reward total was 6.000000. running mean: 0.867928
episode 6724.000000, reward total was 6.000000. running mean: 0.919249
episode 6725.000000, reward total was 2.000000. running mean: 0.930057
episode 6726.000000, reward total was -7.000000. running mean: 0.850756
episode 6727.000000, reward total was -4.000000. running mean: 0.802248
episode 6728.000000, reward total was 8.000000. running mean: 0.874226
episode 6729.000000, reward total was 6.000000. running mean: 0.925484
episode 6730.000000, reward total was 7.000000. running mean: 0.986229
episode 6731.000000, reward total was -1.000000. running mean: 0.966367
episode 6732.000000, reward total was 7.000000. running mean: 1.026703
episode 6733.000000, reward total was -5.000000. running mean: 0.966436
episode 6734.000000, reward total was 3.000000. running mean: 0.986772
episode 6735.000000, reward total was -2.000000. running mean: 0.956904
episode 6736.000000, reward total was 4.000000. running mean: 0.987335
e

episode 6838.000000, reward total was -1.000000. running mean: 0.908612
episode 6839.000000, reward total was -4.000000. running mean: 0.859526
episode 6840.000000, reward total was -3.000000. running mean: 0.820931
episode 6841.000000, reward total was 9.000000. running mean: 0.902722
episode 6842.000000, reward total was 1.000000. running mean: 0.903694
episode 6843.000000, reward total was 2.000000. running mean: 0.914657
episode 6844.000000, reward total was 6.000000. running mean: 0.965511
episode 6845.000000, reward total was 10.000000. running mean: 1.055856
episode 6846.000000, reward total was -1.000000. running mean: 1.035297
episode 6847.000000, reward total was -2.000000. running mean: 1.004944
episode 6848.000000, reward total was -3.000000. running mean: 0.964895
episode 6849.000000, reward total was 1.000000. running mean: 0.965246
episode 6850.000000, reward total was 8.000000. running mean: 1.035593
episode 6851.000000, reward total was -2.000000. running mean: 1.00523

episode 6953.000000, reward total was -2.000000. running mean: 1.100375
episode 6954.000000, reward total was -7.000000. running mean: 1.019371
episode 6955.000000, reward total was -9.000000. running mean: 0.919178
episode 6956.000000, reward total was 11.000000. running mean: 1.019986
episode 6957.000000, reward total was 9.000000. running mean: 1.099786
episode 6958.000000, reward total was -7.000000. running mean: 1.018788
episode 6959.000000, reward total was -2.000000. running mean: 0.988600
episode 6960.000000, reward total was 9.000000. running mean: 1.068714
episode 6961.000000, reward total was -11.000000. running mean: 0.948027
episode 6962.000000, reward total was -1.000000. running mean: 0.928547
episode 6963.000000, reward total was -2.000000. running mean: 0.899261
episode 6964.000000, reward total was -2.000000. running mean: 0.870269
episode 6965.000000, reward total was -3.000000. running mean: 0.831566
episode 6966.000000, reward total was 5.000000. running mean: 0.8